This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [10]:
# If haven't downloaded it yet, please get the data file with wget
#!pip install wget
#import wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/ --no-check-certificate

--2021-11-16 09:24:22--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2021-11-16 09:24:22--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc2d9ed5899ddf73d11c2f87c54.dl.dropboxusercontent.com/cd/0/inline/BaEFym9RbLYA3CMyqs5HTvdlCHRFnhalqv6BlOpOcKgRDANcVg5SLKC-vowtqyFTIjcNQL8osjRsKhYASlYkvUE1f7CuUDiZx1szFiIXmb2raF-y0pGDG-_-HHBYGhojVsno6kbMPh7GyVKYgDHLdgnc/file# [following]
--2021-11-16 09:24:22--  https://ucc2d9ed5899ddf73d11c2f87c54.dl.dropboxusercontent.com/cd/0/inline/BaEFym9RbLYA3CMyqs5HTvdlCHRFnhalqv6BlOpOcKgRDANcVg5SLKC-

 14900K .......... .......... .......... .......... .......... 46% 8.07M 3s
 14950K .......... .......... .......... .......... .......... 46% 4.64M 3s
 15000K .......... .......... .......... .......... .......... 46% 7.66M 3s
 15050K .......... .......... .......... .......... .......... 46% 6.44M 3s
 15100K .......... .......... .......... .......... .......... 46% 7.71M 3s
 15150K .......... .......... .......... .......... .......... 46% 6.45M 3s
 15200K .......... .......... .......... .......... .......... 47% 5.95M 3s
 15250K .......... .......... .......... .......... .......... 47% 1.04M 3s
 15300K .......... .......... .......... .......... .......... 47% 9.52M 3s
 15350K .......... .......... .......... .......... .......... 47% 5.17M 3s
 15400K .......... .......... .......... .......... .......... 47% 1.09M 3s
 15450K .......... .......... .......... .......... .......... 47% 8.13M 3s
 15500K .......... .......... .......... .......... .......... 47% 5.47M 3s
 15550K ....

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd

data = pd.read_csv('data/data_000637.txt') #importing the dataframe
total_rows = data.shape[0] #getting the total number of rows in the file

N = 500000 #value bigger than 10k and lower than total_rows

data = pd.read_csv('data/data_000637.txt', nrows = N) #resetting the dataframe to N rows

data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
499995,1,1,25,3869205221,2330,28
499996,1,1,23,3869205221,2335,10
499997,1,1,139,3869205221,2336,0
499998,1,1,24,3869205221,2333,21


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [3]:
orbit_cnt = data[["ORBIT_CNT","BX_COUNTER"]]
#orbit_cnt
things = pd.DataFrame(orbit_cnt).to_numpy()
#The orbit_cnt variable is increased every x bx_counter
# x = bx_counter(orbit_cnt+1) - bx_counter(orbit_cnt)
import numpy as np
#set initial values
orb = things[0,0]
bx = things[0,1]
x = []
print("First values: ",orb, bx)
for i in range(N):
    o = things[i,0] #get the orbit counter value
    b = things[i,1] #get the bx counter value
    
    if orb != o:
        #print("At step ",i)
        #print("previous orb value ",orb, "has increased to ", o)
        #print("The number x is the difference between ",b, "and ",bx, " : ",b-bx)
        x.append(b - bx)
        orb = o
        bx = b
        #print(orb,bx)
    else:
        continue

#print(x)
y = np.mean(np.array(x))
print(y)

First values:  3869200167 2374
-0.4402609506057782


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [14]:
time = 0
tdc = data["TDC_MEAS"]
#print(tdc)
tdcnp = pd.DataFrame(tdc).to_numpy()

for i in range(N):
    time = time + int(tdcnp[i])*25 + int(things[i,1])*25

print(time)
time = time*(pow(10,-9))
print(time)

22558577275
22.558577275


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel